In [1]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Cargar el modelo preentrenado
model = RobertaForSequenceClassification.from_pretrained('BSC-TeMU/roberta-base-biomedical-es', num_labels=2)

# Cargar el tokenizador
tokenizer = RobertaTokenizer.from_pretrained('BSC-TeMU/roberta-base-biomedical-es')


C:\Users\serma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\serma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-biomedical-es and are newly initialized: ['c

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar el dataset desde el archivo CSV
data = pd.read_csv('../datasets2/data.csv')

# Dividir el dataset en entrenamiento y prueba (80% entrenamiento, 20% prueba)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Guardar los conjuntos divididos en archivos CSV temporales
train_data.to_csv('train_temp.csv', index=False)
test_data.to_csv('test_temp.csv', index=False)


In [5]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Cargar el modelo preentrenado
model = RobertaForSequenceClassification.from_pretrained('BSC-TeMU/roberta-base-biomedical-es', num_labels=2)

# Cargar el tokenizador
tokenizer = RobertaTokenizer.from_pretrained('BSC-TeMU/roberta-base-biomedical-es')



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-biomedical-es and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from datasets import load_dataset

# Cargar el dataset desde los archivos CSV temporales
dataset = load_dataset('csv', data_files={'train': 'train_temp.csv', 'test': 'test_temp.csv'})

# Función de tokenización
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenizar el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Generating train split: 80 examples [00:00, 2719.64 examples/s]
Generating test split: 20 examples [00:00, 3994.96 examples/s]
Map: 100%|██████████| 20/20 [00:00<00:00, 102.28 examples/s]


In [9]:
import os

# Define los directorios
output_dir = './results'
logging_dir = './logs'

# Crea los directorios si no existen
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)


In [10]:
from transformers import Trainer, TrainingArguments

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir=logging_dir
)

# Definir el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)


C:\Users\serma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer.train()


FailedPreconditionError: ./logs is not a directory